# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)
pd.reset_option("^display")

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
df = pd.read_csv('/Users/TekendraMBP/Documents/Lede_Program/Foundations/10-homework/scraped/hm.csv')

In [3]:
df

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=134...
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=350...
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=724...
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=589...
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=742...
5,Slub-weave Cushion Cover,$3.99 $9.99,http://www.hm.com/us/product/70965?article=709...
6,Braided Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/62818?article=628...
7,Jacquard-weave Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/69163?article=691...
8,Scented Candle in Glass Holder,$9.99 $17.99,http://www.hm.com/us/product/40910?article=409...
9,2-pack Curtain Panels,$27.99 $34.99,http://www.hm.com/us/product/69699?article=696...


In [4]:
#Wow what an easy way to extract the price discount. Neat $ signs
df['price_discounted'] = df['price'].str.extract(r'(\$.*) ' )

In [5]:
#wow what a neat way to extract the original price. more neat dollar signs
df['price_original'] = df['price'].str.extract(r' (\$.*)')

In [6]:
df

,name,price,url,price_discounted,price_original
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=134...,$59.99,$129
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=350...,$6.99,$17.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=724...,$7.99,$17.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=589...,$6.99,$12.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=742...,$17.99,$24.99
5,Slub-weave Cushion Cover,$3.99 $9.99,http://www.hm.com/us/product/70965?article=709...,$3.99,$9.99
6,Braided Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/62818?article=628...,$7.99,$17.99
7,Jacquard-weave Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/69163?article=691...,$7.99,$17.99
8,Scented Candle in Glass Holder,$9.99 $17.99,http://www.hm.com/us/product/40910?article=409...,$9.99,$17.99
9,2-pack Curtain Panels,$27.99 $34.99,http://www.hm.com/us/product/69699?article=696...,$27.99,$34.99


In [7]:
df['article_id'] = df['url'].str.extract(r'(\d\d\d\d\d-[A-Z]$)')

In [8]:
#Turns out those neat ways werent so neat and I need to extract the numbericals from those 
#new columns and make more new columns
df['discount_numerical'] = df['price_discounted'].str.extract(r'(\d.*)')
df['original_price_numerical'] = df['price_original'].str.extract(r'(\d.*)')

In [9]:
#Now I have to convert those into floats
df['discount_numerical'] = df['discount_numerical'].astype(float)

In [10]:
df['original_price_numerical'] = df['original_price_numerical'].astype(float)

In [11]:
df.dtypes

name                         object
price                        object
url                          object
price_discounted             object
price_original               object
article_id                   object
discount_numerical          float64
original_price_numerical    float64
dtype: object

In [12]:
#At least the math works now : )

(df['discount_numerical'] - df['original_price_numerical']) / df['original_price_numerical'] * 100

0    -53.496124
1    -61.145081
2    -55.586437
3    -46.189376
4    -28.011204
5    -60.060060
6    -55.586437
7    -55.586437
8    -44.469150
9    -20.005716
10   -50.083472
11   -50.083472
12   -23.094688
13   -20.004001
14   -20.004001
15   -55.586437
16   -33.344448
17   -50.083472
18   -55.586437
19   -40.016006
20   -60.024010
21   -38.491147
22   -55.586437
23   -50.050050
24   -23.094688
25   -55.586437
26   -42.918455
27   -55.586437
28   -57.224607
29   -23.094688
30   -57.159188
31   -55.586437
32   -50.010002
33   -62.507813
34   -61.585835
35   -20.008003
36   -50.083472
37   -57.159188
38   -61.145081
39   -61.145081
40   -61.145081
41   -50.251256
42   -44.454545
43   -61.145081
44   -61.585835
45   -55.586437
46   -60.060060
47   -61.145081
48   -61.585835
49   -50.083472
50   -50.050050
51   -46.189376
52   -50.083472
53   -56.257032
54   -55.586437
55   -56.257032
56   -50.083472
57   -40.016006
58   -53.887606
59   -27.793218
dtype: float64

In [13]:
#now to save my math as a column

df["price_change"] = (df['discount_numerical'] - df['original_price_numerical']) / df['original_price_numerical'] * 100

In [14]:
#I should delete those price columns and whatever I dont want to use

del df['price_discounted']

In [15]:
del df['price_original']

In [16]:
del df['price']

In [17]:
df

,name,url,article_id,discount_numerical,original_price_numerical,price_change
0,Washed Linen Duvet Cover Set,http://www.hm.com/us/product/13472?article=134...,13472-N,59.99,129.00,-53.496124
1,Candle in Glass Jar,http://www.hm.com/us/product/35079?article=350...,35079-D,6.99,17.99,-61.145081
2,Glittery Cushion Cover,http://www.hm.com/us/product/72462?article=724...,72462-A,7.99,17.99,-55.586437
3,Textured-weave Cushion Cover,http://www.hm.com/us/product/58926?article=589...,58926-C,6.99,12.99,-46.189376
4,Stoneware Bowl,http://www.hm.com/us/product/74242?article=742...,74242-A,17.99,24.99,-28.011204
5,Slub-weave Cushion Cover,http://www.hm.com/us/product/70965?article=709...,70965-D,3.99,9.99,-60.060060
6,Braided Cushion Cover,http://www.hm.com/us/product/62818?article=628...,62818-B,7.99,17.99,-55.586437
7,Jacquard-weave Cushion Cover,http://www.hm.com/us/product/69163?article=691...,69163-B,7.99,17.99,-55.586437
8,Scented Candle in Glass Holder,http://www.hm.com/us/product/40910?article=409...,40910-C,9.99,17.99,-44.469150
9,2-pack Curtain Panels,http://www.hm.com/us/product/69699?article=696...,69699-B,27.99,34.99,-20.005716


In [18]:
df.to_csv("hm_cleaned.csv")

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [19]:
df = pd.read_csv('/Users/TekendraMBP/Documents/Lede_Program/Foundations/10-homework/scraped/sci-fi.csv')

In [20]:
#We're gonna strip the whitespaces using .strip()
df['title'] = df['title'].str.strip()

In [21]:
#So fresh and so clean.
df['title']

0                                   The Handmaid's Tale
1               The Hunger Games (The Hunger Games, #1)
2                Frankenstein, or The Modern Prometheus
3     A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
4                             The Left Hand of Darkness
5                             Divergent (Divergent, #1)
6                  Catching Fire (The Hunger Games, #2)
7                             The Giver (The Giver, #1)
8                                      The Dispossessed
9                                               Kindred
10                       Oryx and Crake (MaddAddam, #1)
11                    Mockingjay (The Hunger Games, #3)
12                             The Time Traveler's Wife
13               Doomsday Book (Oxford Time Travel, #1)
14                                  The Lathe of Heaven
15                        The Sparrow (The Sparrow, #1)
16              Shards of Honour  (Vorkosigan Saga, #1)
17               Ancillary Justice (Imperial Rad

In [22]:
#I want everything after that first parenthesis, and before that comma seperating the series name
# and the series no. 
#so I called for everything under the sun in the paranthesis that asks for things that I want

df['series'] = df['title'].str.extract(r'\((.*),')

In [23]:
df

,full_rating,full_score,rank,title,url,series
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 peopl...",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 peopl...",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,The Hunger Games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 peopl...",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Pr...,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 peopl...",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,A Wrinkle in Time Quintet
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 peopl...",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,NaN
5,"4.23 avg rating — 2,345,974 ratings","\nscore: 12,935,\n and\n134 peopl...",6,"Divergent (Divergent, #1)",/book/show/13335037-divergent,Divergent
6,"4.30 avg rating — 2,049,239 ratings","\nscore: 12,261,\n and\n128 peopl...",7,"Catching Fire (The Hunger Games, #2)",/book/show/6148028-catching-fire,The Hunger Games
7,"4.12 avg rating — 1,379,452 ratings","\nscore: 11,238,\n and\n117 peopl...",8,"The Giver (The Giver, #1)",/book/show/3636.The_Giver,The Giver
8,"4.19 avg rating — 57,605 ratings","\nscore: 10,246,\n and\n107 peopl...",9,The Dispossessed,/book/show/13651.The_Dispossessed,NaN
9,"4.20 avg rating — 53,473 ratings","\nscore: 9,907,\n and\n104 people...",10,Kindred,/book/show/60931.Kindred,NaN


In [24]:
#I wanted the stuff after that comma [space] hashtag combo. But everything before that last 
# paranthesis. So I called everything under the sun in between those two things

df['series_no'] = df['title'].str.extract(r', #(.*)\)')

In [25]:
df

,full_rating,full_score,rank,title,url,series,series_no
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 peopl...",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,NaN,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 peopl...",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,The Hunger Games,1
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 peopl...",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Pr...,NaN,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 peopl...",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,A Wrinkle in Time Quintet,1
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 peopl...",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,NaN,NaN
5,"4.23 avg rating — 2,345,974 ratings","\nscore: 12,935,\n and\n134 peopl...",6,"Divergent (Divergent, #1)",/book/show/13335037-divergent,Divergent,1
6,"4.30 avg rating — 2,049,239 ratings","\nscore: 12,261,\n and\n128 peopl...",7,"Catching Fire (The Hunger Games, #2)",/book/show/6148028-catching-fire,The Hunger Games,2
7,"4.12 avg rating — 1,379,452 ratings","\nscore: 11,238,\n and\n117 peopl...",8,"The Giver (The Giver, #1)",/book/show/3636.The_Giver,The Giver,1
8,"4.19 avg rating — 57,605 ratings","\nscore: 10,246,\n and\n107 peopl...",9,The Dispossessed,/book/show/13651.The_Dispossessed,NaN,NaN
9,"4.20 avg rating — 53,473 ratings","\nscore: 9,907,\n and\n104 people...",10,Kindred,/book/show/60931.Kindred,NaN,NaN


In [26]:
#avg_rating
#rating_count
#total_score
#score_votes

In [27]:
#All I want is a digit, a period and two more digits. But remember periods a regex thing so
# I gotta ask for it as a period thing. 
df['avg_rating'] = df['full_rating'].str.extract(r'(\d\.\d\d)')

In [28]:
df['rating_count'] = df['full_rating'].str.extract(r'—(.*) ratings' )

In [41]:
df.head(100)

,full_rating,full_score,rank,title,url,series,series_no,avg_rating,rating_count
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 peopl...",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,NaN,NaN,4.07,"785,502"
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 peopl...",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,The Hunger Games,1,4.34,"5,212,935"
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 peopl...",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Pr...,NaN,NaN,3.76,"922,308"
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 peopl...",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,A Wrinkle in Time Quintet,1,4.04,"702,272"
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 peopl...",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,NaN,NaN,4.06,"77,664"
5,"4.23 avg rating — 2,345,974 ratings","\nscore: 12,935,\n and\n134 peopl...",6,"Divergent (Divergent, #1)",/book/show/13335037-divergent,Divergent,1,4.23,"2,345,974"
6,"4.30 avg rating — 2,049,239 ratings","\nscore: 12,261,\n and\n128 peopl...",7,"Catching Fire (The Hunger Games, #2)",/book/show/6148028-catching-fire,The Hunger Games,2,4.30,"2,049,239"
7,"4.12 avg rating — 1,379,452 ratings","\nscore: 11,238,\n and\n117 peopl...",8,"The Giver (The Giver, #1)",/book/show/3636.The_Giver,The Giver,1,4.12,"1,379,452"
8,"4.19 avg rating — 57,605 ratings","\nscore: 10,246,\n and\n107 peopl...",9,The Dispossessed,/book/show/13651.The_Dispossessed,NaN,NaN,4.19,"57,605"
9,"4.20 avg rating — 53,473 ratings","\nscore: 9,907,\n and\n104 people...",10,Kindred,/book/show/60931.Kindred,NaN,NaN,4.20,"53,473"


In [43]:
df['total_score'] = df['full_score'].str.extract(r'\nscore: (.*),\n')

In [46]:
df['score_votes'] = df['full_score'].str.extract(r'and\n(.*) people')

In [49]:
df.to_csv('sci-fi_cleaned.csv')

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [60]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})



In [59]:
filenames

['pdfs/11-11916.h1.pdf.txt',
 'pdfs/12-01601.a1.pdf.txt',
 'pdfs/11-03073.h1.pdf.txt',
 'pdfs/11-04909.h1.pdf.txt',
 'pdfs/11-08313.h1.pdf.txt',
 'pdfs/12-01969.h1.pdf.txt',
 'pdfs/11-02438.h1.pdf.txt',
 'pdfs/11-14135.h1.pdf.txt',
 'pdfs/11-12635.h1.pdf.txt',
 'pdfs/12-00038.h1.pdf.txt',
 'pdfs/11-14832.h1.pdf.txt',
 'pdfs/11-12537.a1.pdf.txt',
 'pdfs/12-01601.h1.pdf.txt',
 'pdfs/11-07728.h1.pdf.txt',
 'pdfs/12-01787.h1.pdf.txt']

display.max_cols: Number of columns to show at once
display.max_rows: Number of rows to show at once
display.max_colwidth: Maximum number of characters displayed from a string

In [111]:
pd.set_option("display.max_colwidth", 2000)

In [112]:
df

,filename,content,lawyer_gov,lawyer_app,judge
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...","David F. Hayes, Esq.",Pro se,"MENDEZ, Francisco"
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",NaN,NaN,NaN
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARIN

Okay, now do the work and **make those new columns!**

In [ ]:
# lawyer_app, the applicant's lawyer (pro se means that they did it themselves, that's fine)
# lawyer_gov, the government's lawyer
# judge, the name of the judge
# access, whether the clearance is granted or denied (although you might miss a few)

In [80]:
df['content'].str.extract(r'FOR GOVERNMENT\n\n(.*)')

,0
0,NaN
1,"James B. Norman, Esq., Chief Department Counsel"
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [91]:
df['lawyer_gov'] = df['content'].str.extract(r'For Government:(.*),',  re.IGNORECASE)

In [92]:
 df

,filename,content,lawyer_gov
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...","David F. Hayes, Esq."
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",NaN
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n 

In [104]:
df['lawyer_app'] = df['content'].str.extract(r'\n\nFor Applicant: (.*)')

In [105]:
df

,filename,content,lawyer_gov,lawyer_app
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...","David F. Hayes, Esq.",Pro se
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",NaN,NaN
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn th

In [108]:
df['judge'] = df['content'].str.extract(r'(.*), Administrative Judge:')

In [127]:
df['access'] = df['content'].str.extract(r'access to classified information is(.*).')

In [128]:
df

,filename,content,lawyer_gov,lawyer_app,judge,access
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense Consolidated Adjudications \nFacility sent Applicant a Statement of Reasons (SOR) alleging that his circumstances \nraised security concerns under the foreign preference and foreign influence guidelines.1 \n \n1 This action was taken under Executive Order (E.O.) 10865, Safeguarding Classified Information within \nIndustry (February 20, 1960), as amended; Department of Defense Directive 5220.6, Defense Industrial \nPersonnel Security Clearance Review Program (January 2, 1992), as amended (Directive); and the \nAdjudicative Guidelines implemented by the Department of Defense on September 1, 2006. \n\nFor Government: David F. Hayes, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\n \n\n \n1 \n \n \n\n On July 26, 2014, Applicant answered the SOR, admitted all the SOR allegations, \n...","David F. Hayes, Esq.",Pro se,"MENDEZ, Francisco",NaN
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, Defense Office of Hearings\nand Appeals (DOHA) Administrative Judge Martin H. Mogul denied Applicant’s request for a\nsecurity clearance. Applicant appealed pursuant to Directive ¶¶ E3.1.28 and E3.1.30.\n\nApplicant raised the following issues on appeal: whether the Judge erred in making findings\nof fact and whether the Judge’s decision was arbitrary, capricious, or contrary to law. Consistent\nwith the following, we affirm.\n\nThe Judge’s Findings of Fact \n\nApplicant is 46 years old, married, and has four children. He earned a bachelor’s degree in\n1994. Since 2010, he has been employed by a defense contractor. In his security clearance\napplication, he explained that he encountered difficultly paying bills in 2008 after being laid off\nfrom a second job due to the company’s bankruptcy. He gave no explanation for why he did not\nbegin attempting to resolve the debts until the issuance of the SOR.\n\nApplicant admitted each of the five delinquent debts alleged in...",NaN,NaN,NaN,NaN
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE O

In [129]:
df.to_csv('court_cleaned.csv')

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [141]:
austen = open('./data/Austen_Pride.txt').read()

## Look at the first 500 or so characters of it 

In [143]:
austen[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [186]:
re.findall(r'\b(s?he)\b', austen, re.IGNORECASE)

['he',
 'he',
 'she',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'she',
 'he',
 'She',
 'she',
 'she',
 'He',
 'he',
 'she',
 'he',
 'She',
 'She',
 'she',
 'she',
 'she',
 'he',
 'he',
 'he',
 'she',
 'she',
 'he',
 'he',
 'she',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'She',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'She',
 'he',
 'he',
 'She',
 'She',
 'she',
 'she',
 'he',
 'he',
 'He',
 'he',
 'he',
 'she',
 'she',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'He',
 'she',
 'She',
 'she',
 'he',
 'He',
 'he',
 'she',
 'He',
 'she',
 'He',
 'he',
 'He',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [187]:
re.findall(r'\b(s?he\b \w+)\b', austen, re.IGNORECASE)

['he is',
 'he had',
 'she told',
 'he came',
 'he agreed',
 'he is',
 'he married',
 'he may',
 'he comes',
 'she ought',
 'he comes',
 'he chooses',
 'she is',
 'She was',
 'she was',
 'she fancied',
 'He had',
 'he should',
 'she had',
 'he suddenly',
 'She has',
 'She is',
 'she times',
 'she will',
 'she will',
 'he continued',
 'he wished',
 'she began',
 'she had',
 'he spoke',
 'he left',
 'he would',
 'he eluded',
 'He was',
 'he meant',
 'He had',
 'he had',
 'he saw',
 'he wore',
 'She could',
 'he could',
 'she began',
 'he might',
 'he ought',
 'he brought',
 'he had',
 'he was',
 'he was',
 'he was',
 'he was',
 'He was',
 'he would',
 'She is',
 'he looked',
 'he withdrew',
 'She is',
 'She told',
 'she had',
 'she had',
 'he was',
 'he had',
 'He had',
 'he had',
 'she entered',
 'she looked',
 'he actually',
 'she was',
 'he asked',
 'he asked',
 'he did',
 'he seemed',
 'she was',
 'he inquired',
 'she was',
 'he danced',
 'he had',
 'he would',
 'he had',
 'He is',
 

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [200]:
re.findall(r'\b(s?he\b) (\w+)\b', austen, re.IGNORECASE)


[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed'),
 ('he', 'is'),
 ('he', 'married'),
 ('he', 'may'),
 ('he', 'comes'),
 ('she', 'ought'),
 ('he', 'comes'),
 ('he', 'chooses'),
 ('she', 'is'),
 ('She', 'was'),
 ('she', 'was'),
 ('she', 'fancied'),
 ('He', 'had'),
 ('he', 'should'),
 ('she', 'had'),
 ('he', 'suddenly'),
 ('She', 'has'),
 ('She', 'is'),
 ('she', 'times'),
 ('she', 'will'),
 ('she', 'will'),
 ('he', 'continued'),
 ('he', 'wished'),
 ('she', 'began'),
 ('she', 'had'),
 ('he', 'spoke'),
 ('he', 'left'),
 ('he', 'would'),
 ('he', 'eluded'),
 ('He', 'was'),
 ('he', 'meant'),
 ('He', 'had'),
 ('he', 'had'),
 ('he', 'saw'),
 ('he', 'wore'),
 ('She', 'could'),
 ('he', 'could'),
 ('she', 'began'),
 ('he', 'might'),
 ('he', 'ought'),
 ('he', 'brought'),
 ('he', 'had'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('He', 'was'),
 ('he', 'would'),
 ('She', 'is'),
 ('he', 'looked'),
 ('he', 'withdrew'),
 ('She', 'is'),
 ('She', 't

In [197]:
words = re.findall(r'\b(s?he\b) (\w+)\b', austen, re.IGNORECASE)

labels = ['pronoun' , 'verb']



## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [211]:
df = pd.DataFrame.from_records(words, columns=labels)


In [199]:
df

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed
5,he,is
6,he,married
7,he,may
8,he,comes
9,she,ought


## How many times is each pronoun used?

In [208]:
df.pronoun.value_counts()

she    1322
he     1054
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed
5,he,is
6,he,married
7,he,may
8,he,comes
9,she,ought


In [213]:
df['pronoun'] = df.pronoun.str.lower()

In [216]:
df.pronoun.value_counts()

she    1647
he     1288
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [217]:
df.verb.value_counts().head(20)

was          372
had          371
could        172
is           139
would         94
has           70
did           67
will          50
might         46
should        41
felt          38
must          37
said          33
saw           32
added         31
thought       31
then          26
replied       22
continued     21
looked        21
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [225]:
df[df.pronoun == "he"].groupby('pronoun').verb.value_counts().head(20)

pronoun  verb   
he       had        166
         was        160
         is          74
         has         49
         could       40
         would       35
         did         29
         should      26
         must        24
         will        24
         might       21
         replied     14
         came        13
         may         13
         said        13
         thought     13
         does        12
         never       12
         meant       11
         then        11
Name: verb, dtype: int64

In [226]:
df[df.pronoun == "she"].groupby('pronoun').verb.value_counts().head(20)

pronoun  verb     
she      was          212
         had          205
         could        132
         is            65
         would         59
         did           38
         felt          33
         saw           29
         will          26
         might         25
         added         23
         has           21
         said          20
         thought       18
         should        15
         then          15
         found         13
         must          13
         soon          12
         continued     11
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [249]:
df[df.verb == "cried"].pronoun.value_counts(normalize=True) * 100

she    91.666667
he      8.333333
Name: pronoun, dtype: float64

False    99.92236
True      0.07764
Name: verb, dtype: float64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [252]:
tolkein = open('./data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt').read()

In [253]:
tolkein[0:500]

'THE FELLOWSHIP OF THE RING\n\n\n\n\nBEING THE FIRST PART OF\n\nTHE LORD OF THE RINGS\n\nBY\n\nJ.R.R. TOLKIEN\n\n\n\n\n\nThree Rings for the Elven-kings under the sky,\n\nSeven for the Dwarf-lords in their halls of stone,\n\nNine for Mortal Men doomed to die,\n\nOne for the Dark Lord on his dark throne\n\nIn the Land of Mordor where the Shadows lie.\n\nOne Ring to rule them all, One Ring to find them,\n\nOne Ring to bring them all and in the darkness bind them\n\nIn the Land of Mordor where the Shadows lie.\n\n\n\n\n\nCONTENTS\n\n\nCOV'

In [275]:
len(re.findall(r'\b(s?he)\b', tolkein, re.IGNORECASE))

3221

In [276]:
words = re.findall(r'\b(s?he\b) (\w+)\b', tolkein, re.IGNORECASE)

labels = ['pronoun' , 'verb']

In [277]:
tolkein_df = pd.DataFrame.from_records(words, columns=labels)


In [278]:
tolkein_df

,pronoun,verb
0,he,had
1,He,was
2,he,wrote
3,he,recorded
4,he,had
5,he,sent
6,he,had
7,he,learned
8,he,recorded
9,he,used


In [279]:
tolkein_df['pronoun'] = df.pronoun.str.lower()

In [282]:
tolkein_df

,pronoun,verb
0,he,had
1,he,was
2,she,wrote
3,he,recorded
4,he,had
5,he,sent
6,he,had
7,he,learned
8,he,recorded
9,she,used


AttributeError: 'DataFrame' object has no attribute 'value'